In [0]:
import math
import holidays as hd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from pandas.plotting import register_matplotlib_converters
from tensorflow_probability import sts


In [0]:

def normalize_around_mean(items):
    mean_temperature = np.nanmean(items, dtype=np.float32)
    return items - mean_temperature


def build_model(average_temperature, holidays, sales, model_name="model"):
    temperature_effect = tfp.sts.LinearRegression(design_matrix=average_temperature, name='temperature_effect')

    holiday_effect = tfp.sts.LinearRegression(design_matrix=holidays, name='holiday_effect')

    month_of_year_effect = sts.Seasonal(
        num_seasons=12,
        num_steps_per_season=[31, 28, 31, 30, 30, 31, 31, 31, 30, 31, 30, 31],
        name='month_of_year_effect')
    day_of_week_effect = tfp.sts.Seasonal(
        num_seasons=7,
        num_steps_per_season=1,
        observed_time_series=sales,
        name='day_of_week_effect')
    residual_level = tfp.sts.Autoregressive(
        order=1,
        observed_time_series=sales, name='residual')
    print("Building model {}".format(model_name))
    return tfp.sts.Sum([temperature_effect,
                        holiday_effect,
                        month_of_year_effect,
                        day_of_week_effect,
                        residual_level],
                       observed_time_series=sales, name=model_name)



In [0]:
# 1.5 years of data
dates = np.arange('2018-01-01', '2019-07-01', dtype='datetime64[D]')
holidays = [np.datetime64(date) for y in range(2018,2020) for date, _ in sorted(hd.Belgium(years=y).items())]
holiday_indicators = np.where(np.isin(dates,holidays), 1, 0).astype(np.float32).reshape(-1, 1)

raw_temperatures = np.array("6,7,9,8,8,6,3,3,4,7,7,6,3,3,6,6,4,7,3,2,2,7,8,12,8,6,4,8,9,6,7,3,3,3,1,0,-1,-1,-1,-1,3,4,2,2,2,6,3,3,2,2,3,1,1,-1,0,-2,-4,-4,-6,-3,-3,2,7,7,8,6,5,7,12,12,10,6,8,7,9,-2,-2,0,3,3,5,5,9,7,7,6,7,7,8,7,6,9,12,10,6,10,16,17,14,14,11,10,11,12,14,12,14,18,21,21,17,20,13,13,12,11,12,12,10,9,9,11,11,13,16,19,19,21,18,12,13,18,12,13,17,15,12,11,10,15,18,17,17,18,19,23,22,23,21,21,22,17,16,19,20,17,21,21,19,19,21,19,16,14,16,18,17,16,17,18,21,14,13,15,16,18,18,21,22,23,23,23,22,23,23,21,22,22,20,20,16,18,19,20,22,23,23,21,19,21,21,22,22,24,26,27,26,27,21,22,24,23,23,24,26,24,22,26,27,20,18,17,17,22,18,19,18,21,18,17,19,21,22,21,18,16,13,14,17,17,16,16,16,15,17,18,20,19,18,14,14,18,18,19,16,15,14,15,17,20,22,19,20,16,12,11,8,10,13,14,12,10,10,9,12,13,14,14,17,12,10,13,16,19,19,20,19,18,18,15,14,13,10,11,12,10,13,12,9,6,4,4,5,8,11,7,4,8,11,15,11,9,9,12,12,10,11,9,9,6,6,6,6,6,6,6,6,5,4,5,7,9,11,10,7,12,12,6,8,11,9,8,8,5,3,2,0,-1,0,2,7,7,8,7,11,10,9,6,1,1,2,0,4,6,8,7,5,4,3,5,4,6,7,4,3,4,7,8,6,6,6,3,0,-1,-2,-2,-2,-1,-1,1,6,5,3,2,0,-2,2,2,3,2,3,5,8,8,8,7,4,3,3,7,10,9,10,9,8,7,9,8,8,8,11,12,12,8,7,8,11,9,7,10,8,8,9,8,5,7,7,8,10,11,7,4,6,9,9,12,9,8,7,8,8,9,11,13,10,10,11,6,6,8,11,13,9,9,8,7,5,4,4,9,12,12,16,18,18,18,19,17,16,12,12,10,8,11,12,11,9,9,6,7,7,9,10,11,12,11,9,11,11,12,12,12,12,12,11,13,14,16,17,13,15,16,12,14,17,17,20,24,18,18,15,13,14,14,14,17,14,14,14,17,17,17,19,21,19,17,16,18,23,26,27,22,19,19,25,23".split(",")).astype(np.float32)
temperatures = normalize_around_mean(raw_temperatures).astype(np.float32).reshape(-1, 1)

sales = np.array("0,91,80,69,75,90,0,66,64,84,66,78,76,0,76,89,63,94,75,68,0,80,83,75,73,80,62,0,72,69,82,85,69,83,0,53,62,72,78,66,84,0,65,56,77,73,80,67,0,80,80,73,58,70,67,0,70,68,43,68,54,83,0,92,76,62,63,76,83,0,77,87,85,87,90,62,0,61,81,77,86,69,72,0,76,76,64,67,91,68,0,0,86,73,84,72,86,0,85,77,82,96,81,77,0,83,87,95,92,112,91,0,88,91,98,84,98,97,0,77,0,95,88,93,98,0,100,93,110,0,93,95,0,105,89,96,97,89,99,0,0,109,97,102,114,123,0,107,105,91,107,96,104,0,117,103,98,109,94,88,0,93,86,79,92,84,104,0,92,106,113,106,103,99,0,100,94,113,100,93,111,0,103,119,122,93,114,95,0,111,108,86,95,111,111,0,97,92,92,116,112,0,0,98,105,125,124,119,111,0,97,118,114,98,98,101,0,112,115,106,96,91,102,0,98,114,0,110,106,96,0,108,117,111,99,89,96,0,101,97,91,96,97,88,0,104,88,114,102,99,103,0,94,95,104,87,93,84,0,50,103,88,106,87,89,0,92,96,94,88,86,81,0,87,84,95,105,92,87,0,84,80,90,97,100,102,0,110,102,99,97,77,99,0,81,88,103,77,85,78,0,72,66,89,0,67,80,0,76,98,81,96,93,88,0,90,71,72,69,77,50,50,50,50,50,50,93,69,0,50,50,79,82,87,68,0,102,78,85,82,80,93,0,84,79,59,64,79,79,0,65,84,94,83,89,100,0,92,0,67,79,75,78,0,92,0,69,84,88,89,0,75,77,86,76,81,67,0,90,69,75,82,66,50,0,69,65,77,65,71,63,0,67,80,71,60,66,79,0,80,60,87,95,76,76,0,88,81,88,73,87,84,0,94,75,79,70,86,73,0,82,78,82,94,79,76,0,97,76,79,97,91,86,0,64,66,73,79,77,91,0,60,76,73,89,99,79,0,89,74,78,83,80,101,0,83,81,88,89,72,82,0,92,76,81,87,90,87,0,85,88,88,81,107,108,0,0,101,93,77,95,76,0,71,92,0,85,97,83,0,66,92,70,95,75,94,0,87,86,77,50,50,50,50,99,100,105,97,93,80,0,103,103,90,0,83,113,0,100,92,98,89,79,101,0,0,84,101,88,110,111,0,95,118,92,89,89,120,0,108,112,81,91,102,101,0".split(",")).astype(np.float32)

num_forecast_steps = 7 * 2 # Two weeks.
sales_training_data = sales[:-num_forecast_steps]

In [4]:
tf.compat.v1.reset_default_graph()

forecast_model = build_model(temperatures, holiday_indicators, sales_training_data, "forecast_model")

with tf.compat.v1.variable_scope('sts_elbo', reuse=tf.compat.v1.AUTO_REUSE):
    variational_loss, variational_distributions = tfp.sts.build_factored_variational_loss(forecast_model,
                                                                                          sales_training_data, seed=42)

train_op = tf.compat.v1.train.AdamOptimizer(0.1).minimize(variational_loss)

num_variational_steps = 201
num_variational_steps = int(num_variational_steps)

with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())

    for i in range(num_variational_steps):
        _, loss_ = sess.run((train_op, variational_loss))
        if i % 20 == 0:
            print("step {} -ELBO {}".format(i, loss_))

    q_samples_sales_ = sess.run({k: q.sample(50) for k, q in variational_distributions.items()})

print("Inferred parameters:")
for param in forecast_model.parameters:
    print("{}: {} +- {}".format(param.name,
                                np.mean(q_samples_sales_[param.name], axis=0),
                                np.std(q_samples_sales_[param.name], axis=0)))


W0717 13:07:38.716151 139977087596416 linear_operator.py:940] Using (possibly slow) default implementation of to_dense.  Converts by self.matmul(identity).
W0717 13:07:38.933471 139977087596416 linear_operator.py:940] Using (possibly slow) default implementation of to_dense.  Converts by self.matmul(identity).
W0717 13:07:39.112634 139977087596416 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_probability/python/internal/distribution_util.py:1846: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Building model forecast_model
step 0 -ELBO 15114.6337890625
step 20 -ELBO 4607.3525390625
step 40 -ELBO 3422.296142578125
step 60 -ELBO 3046.76220703125
step 80 -ELBO 2856.23046875
step 100 -ELBO 2723.970703125
step 120 -ELBO 2649.5126953125
step 140 -ELBO 2572.2744140625
step 160 -ELBO 2533.410400390625
step 180 -ELBO 2491.953125
step 200 -ELBO 2450.662841796875
Inferred parameters:
observation_noise_scale: 5.790980815887451 +- 0.026065193116664886
temperature_effect/_weights: [1.3669882] +- [0.04401274]
holiday_effect/_weights: [-9.726252] +- [0.4846987]
month_of_year_effect/_drift_scale: 2.151461362838745 +- 1.791658878326416
day_of_week_effect/_drift_scale: 5.284224033355713 +- 0.20885145664215088
residual/_coefficients: [-0.13859378] +- [0.10961989]
residual/_level_scale: 6.701646327972412 +- 0.07875201851129532


In [0]:

sales_forecast_dist = tfp.sts.forecast(
    model=forecast_model,
    observed_time_series=sales_training_data,
    parameter_samples=q_samples_sales_,
    num_steps_forecast=num_forecast_steps)

In [0]:
num_samples = 10

with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    sales_forecast_mean = sess.run(sales_forecast_dist.mean()[..., 0])
    sales_forecast_scale = sess.run(sales_forecast_dist.stddev()[..., 0])
    sales_forecast_samples = sess.run(sales_forecast_dist.sample(num_samples)[..., 0])


In [0]:
sales_forecast_scale 